决策树模型实例

作者：谢文伟

邮件：jim.xie.cn@outlook.com

主页：https://github.com/jim-xie-cn/ai-cv

In [ ]:
#引用以下包做数据处理
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#引用以下包做数据可视化
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from warnings import filterwarnings

In [ ]:
filterwarnings('ignore')
pd.set_option('display.float_format',lambda x:'%.2f'%x) #不使用科学计数法
sns.set(font_scale=1.5) #设置统计图字体大小
plt.rcParams['font.sans-serif']=['SimHei'] #在统计图上显示中文
plt.style.use({'figure.figsize':(24, 8)})  #设置画布大小

In [ ]:
df=pd.read_csv("./dataset/bmi/500_Person_Gender_Height_Weight_Index.csv")
df=df.rename(columns={'Gender':'性别','Height':'身高'})
df=df.rename(columns={'Weight':'体重','Index':'等级'})
df['性别']= df['性别'].str.replace("Male","男")
df['性别']= df['性别'].str.replace("Female","女")

In [ ]:
features=df[['性别','身高','体重']] #使用性别，身高，体重作为输入特征，等级作为预测标签
labels=df['等级']
#使用DictVectorizer将特征转换成训练的输入向量
#sklearn需要numpy格式数据进行训练，现将数据集转化为numpy格式
vec = DictVectorizer(sparse=False)
X =  vec.fit_transform(features.to_dict(orient='records'))
Y = np.array(labels)
feature_names = vec.get_feature_names()
#划分成训练集和测试集，测试集占总样本的10%，训练集占90%，不打乱顺序
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.10,shuffle=False)

In [ ]:
#设置分裂条件，当树的最大深度示15层或节点下样本多于5个停止分裂
#使用C4.5算法训练决策树，如果使用CART算法，将criterion设置cart
dt = DecisionTreeClassifier(criterion='entropy',max_depth=15,min_samples_split=5)
#使用训练集训练模型
dt.fit(X_train,Y_train)
#在测试集上测试预测效果
print("模型得分:",dt.score(X_test,Y_test))

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
#可视化决策树，只显示两层子节点
dot_data = export_graphviz(dt,filled=False,rounded=True,max_depth=2,
                           feature_names=feature_names,
                           class_names=['非常虚弱','虚弱','瘦弱','正常','过重','肥胖'])
graph = graphviz.Source(dot_data)
#保存和显示可视化结果
graph.render(filename ="tree", directory ='./output/', format='plain')
graph.render(filename ="tree", directory ='./output/', format='jpg')
graph